In [4]:
#| default_exp redemet
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# REDEMET

> Este módulo concentra as constantes, funções de carga, processamento, mesclagem e salvamento de dados aeronáuticos provenientes da API do REDEMET

In [7]:
#| export
import os
import json
from datetime import datetime
from urllib.request import urlopen
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

## CONSTANTES


Dados para acesso à API GEOAISWEB

In [8]:
#| export
URL = "https://api-redemet.decea.mil.br/produtos/radar/maxcappi?api_key={}&{}"

In [11]:
#|export
def get_redemet() -> pd.DataFrame:  # DataFrame com frequências, coordenadas e descrição das estações VOR
    # sourcery skip: use-fstring-for-concatenation
    """Faz a requisição get à API do REDEMET usanda a chave `apikey`, processa o json e o retorna como Dataframe"""
    load_dotenv()
    date = datetime.now().strftime("%Y%m%d")
    link = URL.format(os.environ["RMETKEY"], date)
    response = urlopen(link)
    if (
        response.status != 200
        or "application/json" not in response.headers["content-type"]
    ):
        raise ValueError(
            f"Resposta a requisição não foi bem sucedida: {response.status=}"
        )
    data_json = json.loads(response.read())
    df = pd.json_normalize(
        data_json["data"]["radar"][0],
    )
    df["Frequency"] = "2800"
    df["Description"] = "[RMET] " + df.nome.astype("string")
    df = df[["Frequency", "lat_center", "lon_center", "Description"]]
    for c in df.columns:
        df[c] = df[c].astype("string")
    return df.rename(columns={"lat_center": "Latitude", "lon_center": "Longitude"})

In [12]:
#| eval: false
get_redemet()

,Frequency,Latitude,Longitude,Description
0,2800,-16.201531,-40.674153,[RMET] Radar - Almenara/MG
1,2800,-1.406667,-48.461389,[RMET] Radar - Belém/PA
2,2800,2.844166667,-60.70027777778,[RMET] Radar - Boa Vista/RR
3,2800,-31.404,-52.701644,[RMET] Radar - Canguçu/RS
4,2800,-7.595833,-72.767778,[RMET] Radar - Cruzeiro do Sul/AC
5,2800,-15.97643,-48.016142,[RMET] Radar - Gama/DF
6,2800,-20.27855,-54.47396,[RMET] Radar - Jaraguari/MS
7,2800,-3.149216,-59.991881,[RMET] Radar - Manaus/AM
8,2800,-0.047222,-51.097778,[RMET] Radar - Macapá/AP
9,2800,-9.55129,-35.77068,[RMET] Radar - Maceió/AL
